<h1 align='center'><span style='color:green ; font-size:35px ; font-weight:bold'>Segmenting and Clustering Neighborhoods in Toronto</span></h1>

<a  href="https://www.google.com/maps/place/Toronto,+ON,+Canada/"><img src = "http://www.world-guides.com/images/toronto/toronto_map.jpg" width = 300 align="middle"> </a>

***

## Introduction

In this assignment, We are going to build the data frames with postal codes and the neighborhoods
located in Toronto, Canada. and then we are going to build another data frame with postal code and associated Latitude and Longitude data. After that we will merge/join both data frames and then will segment, and cluster the neighborhoods in the city of Toronto.

## Table of Contents

### [Section-Q3 : Segmentation and Clustering the Neighborhoods in Toronto, Canada](#item7)

1. <a href="#item1">Importing the Libraries</a>
2. <a href="#item2">Toronto Postal Code and Neighbourhood and lat long data reading from previous section</a> 
3. <a href="#item8">Creating Map using folium with data available in dataframe </a>
9. <a href="#item9">Getting the neighborhood surrounding venues and details. </a>
10. <a href="#item10">Analyzing Each neighborhood and it's venues. </a>
11. <a href="#item11">Create data frame with top 10 venues in each neighborhood in Toronto, CA </a>
12. <a href="#item12">Clustering the Neighborhoods </a>
13. <a href="#item13">Creating Map with cluster data  </a>
14. <a href="#item14">Examine the cluster data  </a>

<p></p>
<p></p>
<p></p>
<p></p>
<p></p>
<p></p>

<p></p>
<p></p>
<p></p>

# Section-Q3 : Segmentation and Clustering the Neighborhoods in Toronto, Canada

<a id="item1"></a>

## 1. Importing the Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if the geopy package is not available to import.
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#from pandas import json_normalize
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if the folium package is not available to import.
import folium # map rendering library


pv = !python --version
print('Libraries imported.')
print('Python version used in this project', pv)

Libraries imported.
Python version used in this project ['Python 3.6.10 :: Anaconda, Inc.']


<a id="item2"></a>

## 2. Toronto Postal Code, Neighbourhood and Latitude and Longitude data preparation


In this section we used pandas "read_csv" method to read the data from csv file created in previous section. The file has "PostalCode, Borough and Neighborhood' as columns.

In [2]:
df_toronto_pc_nh_latlong = pd.read_csv('toronto_postalcodes_nh_latlong.csv')
df_toronto_pc_nh_latlong.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<a id="item3"></a>

<a id="item7"></a>

#### @ Getting latitude and longitudes for Toronto, Ontario, CANADA

In [3]:
address = 'Toronto, ON, CA'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<a id="item8"></a>

## 3. Creating Map using folium with data available in dataframe (i.e.df_toronto_pc_nh ) 

In [4]:
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(df_toronto_pc_nh_latlong['Latitude'], df_toronto_pc_nh_latlong['Longitude'], df_toronto_pc_nh_latlong['Borough'], df_toronto_pc_nh_latlong['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3196cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### @ Define Foursquare Credentials and Version

In [5]:
CLIENT_ID = 'YWOI1OAAD2SGEXVYQI0SONQQHQYQHXHEOQW4HLTSXZZ5G5IX' # your Foursquare ID
CLIENT_SECRET = 'C3TWFMSQGG0NRRXW2R1S3JUD3ZKKLOK3N1B0HHZTIE2LVEYH' # your Foursquare Secret
#CLIENT_ID = '************************************' # your Foursquare ID
#CLIENT_SECRET = '********************************' # your Foursquare Secret
VERSION = '20200320' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YWOI1OAAD2SGEXVYQI0SONQQHQYQHXHEOQW4HLTSXZZ5G5IX
CLIENT_SECRET:C3TWFMSQGG0NRRXW2R1S3JUD3ZKKLOK3N1B0HHZTIE2LVEYH



<span style='color:red ; font-size:15px ; font-weight:bold'> Note: Removed the credentials after all the sections are executed.</span>

#### @ Creating function "getNearbyVenues" to get the surrounding venues using "explore" endpoint.

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## 4. Getting the neighborhood surrounding venues and details.

In [7]:
toronto_ca_venues = getNearbyVenues(names=df_toronto_pc_nh_latlong['Neighborhood'],
                                   latitudes=df_toronto_pc_nh_latlong['Latitude'],
                                   longitudes=df_toronto_pc_nh_latlong['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South, Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Vill

#### @ Let's check the size of the resulting dataframe and first 5 records.

In [8]:
toronto_ca_venues.shape

(2226, 7)

In [9]:
toronto_ca_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


#### @ Let's check how many venues were returned for each neighborhood

In [10]:
toronto_ca_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14


#### @ Let's check how many unique categories are available in toronto neighborhood venues.

In [11]:
print('There are {} uniques categories in Toronto Area.'.format(len(set(toronto_ca_venues['Venue Category']))))

There are 273 uniques categories in Toronto Area.


<a id="item10"></a>

## 5. Analyzing Each neighborhood and it's venues.

In [12]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_ca_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_ca_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [13]:
toronto_onehot.shape

(2226, 273)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [14]:
toronto_nh_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_nh_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

#### @ Let's print each neighborhood along with the top 5 most common venues

In [15]:
num_top_venues = 5

for hood in toronto_nh_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_nh_grouped[toronto_nh_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1  Latin American Restaurant   0.2
2               Skating Rink   0.2
3             Breakfast Spot   0.2
4                     Lounge   0.2


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1           Gym  0.11
2          Pool  0.11
3  Skating Rink  0.11
4      Pharmacy  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.11
1                Coffee Shop  0.11
2  Middle Eastern Restaurant  0.05
3                 Restaurant  0.05
4                Gas Station  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1          Restaurant  0.08
2      Sand

4           Pizza Place   0.1


----St. James Town----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.05
2          Restaurant  0.05
3               Hotel  0.04
4  Italian Restaurant  0.04


----St. James Town, Cabbagetown----
         venue  freq
0   Restaurant  0.07
1       Bakery  0.07
2  Coffee Shop  0.07
3  Pizza Place  0.05
4         Café  0.05


----Steeles West, L'Amoreaux West----
                  venue  freq
0  Fast Food Restaurant  0.17
1    Chinese Restaurant  0.17
2              Pharmacy  0.08
3                  Bank  0.08
4        Sandwich Place  0.08


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3  American Restaurant  0.05
4               Bakery  0.05


----Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
                 venue  freq
0          Coffee Shop  0.12
1                  Pub  0.12
2  Fried Chicken Joint  0.06
3

<a id="item11"></a>

## 6.Create data frame with top 10 venues in each neighborhood in Toronto, CA

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_nhoods_venues_sorted = pd.DataFrame(columns=columns)
toronto_nhoods_venues_sorted['Neighborhood'] = toronto_nh_grouped['Neighborhood']

for ind in np.arange(toronto_nh_grouped.shape[0]):
    toronto_nhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_nh_grouped.iloc[ind, :], num_top_venues)

toronto_nhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Clothing Store,Lounge,Skating Rink,Eastern European Restaurant,Dumpling Restaurant,Electronics Store,Drugstore,Department Store
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pub,Skating Rink,Pool,Pharmacy,Sandwich Place,Diner,Deli / Bodega
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Supermarket,Sushi Restaurant,Ice Cream Shop,Deli / Bodega,Middle Eastern Restaurant,Restaurant,Pizza Place,Pharmacy
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Indian Restaurant,Fast Food Restaurant,Ice Cream Shop,Juice Bar,Sushi Restaurant,Comfort Food Restaurant


<a id="item12"></a>

## 7. Clustering the Neighborhoods

In [18]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_nh_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_.astype(int)
# to change use .astype() 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 3, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 2])

#### @ Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [19]:
# add clustering labels
toronto_nhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_nh_cluster_data = df_toronto_pc_nh_latlong

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_nh_cluster_data = toronto_nh_cluster_data.join(toronto_nhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_nh_cluster_data.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Financial or Legal Service,Portuguese Restaurant,Hockey Arena,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Bakery,Pub,Park,Theater,Restaurant,Café,Mexican Restaurant,Event Space,Chocolate Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Boutique,Event Space,Shoe Store,Furniture / Home Store,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Concert Hall
4,M7A,Queen's Park / Ontario Provincial Government,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Park,Bank,Beer Bar,Boutique,Seafood Restaurant,Sandwich Place,Burger Joint,Restaurant,Burrito Place


#### @ droping the records where there is no data and converting the cluster label to int data type.

In [20]:
toronto_nh_cluster_data=toronto_nh_cluster_data.dropna()
toronto_nh_cluster_data['Cluster_Labels'] = toronto_nh_cluster_data.Cluster_Labels.astype(int)
toronto_nh_cluster_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Financial or Legal Service,Portuguese Restaurant,Hockey Arena,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Theater,Restaurant,Café,Mexican Restaurant,Event Space,Chocolate Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Accessories Store,Boutique,Event Space,Shoe Store,Furniture / Home Store,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Concert Hall
4,M7A,Queen's Park / Ontario Provincial Government,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Park,Bank,Beer Bar,Boutique,Seafood Restaurant,Sandwich Place,Burger Joint,Restaurant,Burrito Place


<a id="item13"></a>

## 8. Creating Map with cluster data 

In [21]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow.reverse()    # to assign red color to cluster0

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_nh_cluster_data['Latitude'], toronto_nh_cluster_data['Longitude'], toronto_nh_cluster_data['Neighborhood'], toronto_nh_cluster_data['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)

#adding cluster Legend in top left corner of the map
lgd_txt = '<span style="color: {col};">{txt}</span>'
for cl in rainbow:
    folium.FeatureGroup(name= lgd_txt.format( txt= ' Cluster : ' + str(rainbow.index(cl)), col= cl)).add_to(map_clusters)

folium.map.LayerControl('topleft', collapsed= False).add_to(map_clusters)  

map_clusters

<a id="item14"></a>

## 9. Examine the Cluster Data

#### @ Cluster-1

In [22]:
toronto_nh_cluster_data.loc[toronto_nh_cluster_data['Cluster_Labels'] == 0, toronto_nh_cluster_data.columns[[1] + list(range(5, toronto_nh_cluster_data.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Coffee Shop,Financial or Legal Service,Portuguese Restaurant,Hockey Arena,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Theater,Restaurant,Café,Mexican Restaurant,Event Space,Chocolate Shop
3,North York,0,Clothing Store,Accessories Store,Boutique,Event Space,Shoe Store,Furniture / Home Store,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Concert Hall
4,Queen's Park / Ontario Provincial Government,0,Coffee Shop,Park,Bank,Beer Bar,Boutique,Seafood Restaurant,Sandwich Place,Burger Joint,Restaurant,Burrito Place
6,Scarborough,0,Fast Food Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
7,North York,0,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Café,Baseball Field,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store
8,East York,0,Pizza Place,Gastropub,Café,Athletics & Sports,Intersection,Bus Line,Pharmacy,Breakfast Spot,Fast Food Restaurant,Gym / Fitness Center
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Bubble Tea Shop,Bookstore,Diner
10,North York,0,Park,Pub,Sushi Restaurant,Japanese Restaurant,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
11,Etobicoke,0,Gift Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop


#### @ Cluster-2

In [23]:
toronto_nh_cluster_data.loc[toronto_nh_cluster_data['Cluster_Labels'] == 1, toronto_nh_cluster_data.columns[[1] + list(range(5, toronto_nh_cluster_data.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,1,Garden,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Deli / Bodega


#### @ Cluster-3

In [24]:
toronto_nh_cluster_data.loc[toronto_nh_cluster_data['Cluster_Labels'] == 2, toronto_nh_cluster_data.columns[[1] + list(range(5, toronto_nh_cluster_data.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,2,Cafeteria,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,College Rec Center


#### @ Cluster-4

In [25]:
toronto_nh_cluster_data.loc[toronto_nh_cluster_data['Cluster_Labels'] == 3, toronto_nh_cluster_data.columns[[1] + list(range(5, toronto_nh_cluster_data.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Park,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
21,York,3,Park,Women's Store,Market,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
35,East YorkEast Toronto,3,Park,Convenience Store,Metro Station,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio
40,North York,3,Park,Airport,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
49,North York,3,Park,Bakery,Construction & Landscaping,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
64,York,3,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Department Store
66,North York,3,Park,Convenience Store,Bank,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
77,Etobicoke,3,Park,Pizza Place,Sandwich Place,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
91,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
98,Etobicoke,3,River,Park,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio


#### @ Cluster-5

In [26]:
toronto_nh_cluster_data.loc[toronto_nh_cluster_data['Cluster_Labels'] == 4, toronto_nh_cluster_data.columns[[1] + list(range(5, toronto_nh_cluster_data.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,4,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant
